In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
  #  for filename in filenames:
        # print(os.path.join(dirname, filename))
   #     pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
sample_submission_file_path = '../input/herbarium-2022-fgvc9/sample_submission.csv'
sample_submission = pd.read_csv(sample_submission_file_path)
sample_submission.head()

In [ ]:
import json
import operator

train_metadata_file_path = '../input/herbarium-2022-fgvc9/train_metadata.json'
f = open(train_metadata_file_path,)
data = json.load(f)

In [ ]:
annotations = data['annotations']
df = pd.DataFrame(annotations)
df['category_id'] = df['category_id'].map(str)
    
df.head()

In [ ]:
def get_file_path(row):
    image_id = row['image_id']
    part_1 = '../input/herbarium-2022-fgvc9/train_images'
    part_2 = image_id[:3]
    part_3 = image_id[3:5]
    part_4 = f'{image_id}.jpg'
    file_path = os.path.join(part_1, part_2, part_3, part_4)
    
    return file_path

In [ ]:
df['file_path'] = df.apply(lambda row: get_file_path(row), axis=1)
df.head()

In [ ]:
train_df = df.sample(frac=0.8, random_state=200) 
validation_df = df.drop(train_df.index)

In [ ]:
from keras_preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1.0/255.0)
train_generator = \
    datagen.flow_from_dataframe(dataframe=train_df, x_col='file_path', y_col='category_id', batch_size=32, 
                                seed=42, shuffle=True, class_mode="categorical")

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, Activation, MaxPooling2D, Dropout
from keras.layers import Dense, Flatten
from keras import layers

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(256, 256, 3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(15330, activation='softmax'))

model.compile(optimizer='rmsprop', 
              loss="categorical_crossentropy", 
              metrics=["accuracy"])

In [ ]:
valid_generator = \
    datagen.flow_from_dataframe(dataframe=validation_df, x_col='file_path', y_col='category_id', 
                                batch_size=32, seed=42, shuffle=True, class_mode="categorical")

In [ ]:
STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n // valid_generator.batch_size

model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10)

In [ ]:
model.save('model.h5')

In [ ]:
STEP_SIZE_TEST = test_generator.n // test_generator.batch_size

In [ ]:
submission_df.to_csv('submission.csv', index=False)